In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [16]:
def conjugateGradientPreconditioned(A, b, x, i_max=1e3, epsilon=1e-6):
    i = 0
    r = b - A.dot(x)
    d = precondition(A, r) # M_inverse * r
    delta_new = r.T.dot(d)
    delta_0 = delta_new
    X = []; X.append(x)
    while (i < i_max) & (delta_new > epsilon ** 2 * delta_0):
        q = A.dot(d)
        alpha = delta_new / d.T.dot(q)
        x = x + alpha * d
        if i % 50 == 0:
            r = b - A.dot(x)
        else:
            r = r - alpha * q
        s = precondition(A, r)
        delta_old = delta_new
        delta_new = r.T.dot(s)
        beta = delta_new / delta_old
        d = s + beta * d
        X.append(x)
        i += 1
    return x, np.array(X), i

def conjugateGradient(A, b, x, i_max=1e3, epsilon=1e-6):
    i = 0
    r = b - A.dot(x)
    d = r
    delta_new = np.linalg.norm(r) ** 2
    delta_0 = delta_new
    X = []; X.append(x)
    while (i < i_max) & (delta_new > epsilon ** 2 * delta_0):
        q = A.dot(d)
        alpha = delta_new / (d.T.dot(q))
        x = x + alpha * d
        if i % 50:
            r = b - A.dot(x)
        else:
            r = r - alpha * q
        delta_old = delta_new
        delta_new = np.linalg.norm(r) ** 2
        beta = delta_new / delta_old
        d = r + beta * d
        X.append(x)
        i += 1
    return x, np.array(X), i

def precondition(A, r):
    M_inv_r = []
    for i, a_ii in enumerate(A.diagonal()):
        M_inv_r.append(1 / a_ii * r[i])
    return np.array(M_inv_r)

def f(A, b, c, x):
    return 1 / 2 * x.T.dot(A.dot(x)) - b.T.dot(x) + c

In [17]:
A = np.array([[3, 2], [2, 6]])
b = np.array([[2], [-8]])
c = 0
x_0 = np.array([[-2], [-2]])

In [21]:
x, X, n_steps = conjugateGradientPreconditioned(A, b, x_0)
# x, X, n_steps = conjugateGradient(A, b, x_0)
print('x =', x)
print('no. of iterations =', n_steps)

fmin = f(A, b, c, x)
print('Minimum value of function =', fmin)

x = [[ 2.]
 [-2.]]
no. of iterations = 2
Minimum value of function = [[-10.]]


In [22]:
u = np.arange(-4, 6, .5)
v = np.arange(-6, 4, .5)
U, V = np.meshgrid(u, v)

ws = []
for arr in zip(np.ravel(U), np.ravel(V)):
    vec = np.array(arr).T
    ws.append(f(A, b, c, vec))
W = np.array(ws).reshape(U.shape)
    
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(U, V, W)

# ax.set_xlabel('X1')
# ax.set_ylabel('X2')
# ax.set_zlabel('f(x)')

# plt.show()

fig, ax = plt.subplots(1, 1)
cp = ax.contourf(U, V, W)
fig.colorbar(cp)
ax.plot(X[:, 0], X[:, 1], c='r', marker='o')

ax.set_xlabel('X1')
ax.set_ylabel('X2')

plt.show()